In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/eyes',
    'dst_dir': './eyes'
})

### Convert  dataset from brain-dir to ready-dir 

In [3]:
df = load_data.df_fromdir_eyes(params.src_dir, teaching_file='label.tsv')

In [4]:
load_data.classed_dir_fromdf(df, "ready/eyes")

### Load dataset via ready-dir

In [5]:
eyes_dataset = ClassificationDataset('ready/eyes')

In [6]:
eyes_dataset.counts_train_data()

{'異常あり': 208, '異常なし': 517}

In [7]:
aug_eyes = AugmentDataset(eyes_dataset)

Initialised with 725 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/train/output.

### Search best condition of data augmentation

In [8]:
aug_eyes.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/train' 
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

            name label             handle  \
0   113_0131.JPG  異常なし  異常なし/113_0131.JPG   
1   113_0007.JPG  異常なし  異常なし/113_0007.JPG   
2   148_0085.JPG  異常なし  異常なし/148_0085.JPG   
3   143_0103.JPG  異常なし  異常なし/143_0103.JPG   
4   148_0014.JPG  異常なし  異常なし/148_0014.JPG   
5   113_0096.JPG  異常なし  異常なし/113_0096.JPG   
6   184_0171.JPG  異常なし  異常なし/184_0171.JPG   
7   105_0045.JPG  異常なし  異常なし/10

--------------------------------------------------------------------------------
Operations: 4
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: CropPercentage (probability=1 percentage_area=0.8 centre=False randomise_percentage_area=False )
	2: Resize (probability=1 width=96 height=96 resample_filter=BICUBIC )
	3: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.
Epoch 1/10
 - 4s - loss: 0.6618 - acc: 0.7045 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/10
 - 1s - loss: 0.5829 - acc: 0.7415 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/10
 - 1s - loss: 0.6326 - acc: 0.6918 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
 - 1s - loss: 0.6305 - acc: 0.6960 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
 - 1s - loss: 0.5859 - acc: 0.7443 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/10
 - 1s - loss: 0.6035 - acc: 0.7188 - val_loss: 8.0590 - val_acc:

Epoch 3/10
 - 1s - loss: 0.6268 - acc: 0.6960 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
 - 1s - loss: 0.6231 - acc: 0.6932 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
 - 1s - loss: 0.6169 - acc: 0.7031 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/10
 - 1s - loss: 0.5962 - acc: 0.7216 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
 - 1s - loss: 0.5997 - acc: 0.7202 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/10
 - 1s - loss: 0.5988 - acc: 0.7273 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 9/10
 - 1s - loss: 0.6468 - acc: 0.6577 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 10/10
 - 1s - loss: 0.5905 - acc: 0.7287 - val_loss: 8.0590 - val_acc: 0.5000
Test accuracy: 0.5
--------------------------------------------------------------------------------
Operations: 2
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: Shear (probability=0.3 max_shear_left=2 max_shear_right=2 )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_opera

### Execute data augmentation with best condition

In [52]:
import shutil
d = os.path.join(aug_brain.dataset.train_dir, 'output')
if os.path.exists(d):
    shutil.rmtree(d)

In [10]:
aug_eyes.augment_dataset()

Processing 173_0116.JPG:   0%|          | 2/832 [00:00<01:16, 10.78 Samples/s]

Initialised with 517 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/train/異常なし/output.

Processing 108_0189.JPG:   0%|          | 2/832 [00:00<01:08, 12.06 Samples/s]  

Initialised with 208 image(s) found.
Output directory set to /home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/train/異常あり/output.

### Tuning hyper parameter of CNN

In [ ]:
best_condition, best_model = use_hyperas.exec_hyperas(
    aug_eyes.augmented_dir,
    aug_eyes.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=10,
    optimizer='adam'
)
fname = 'simplenet.eyes.hdf5'
best_model.save(fname)

def data():
    train_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/auged' 
    test_dir = '/home/fytroo/Tuner/dev/standard_datasets/5a66d177b037cb1b1503a789/validation'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir_classed(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir_classed(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

            name label             handle  \
0   113_0131.JPG  異常なし  異常なし/113_0131.JPG   
1   113_0007.JPG  異常なし  異常なし/113_0007.JPG   
2   148_0085.JPG  異常なし  異常なし/148_0085.JPG   
3   143_0103.JPG  異常なし  異常なし/143_0103.JPG   
4   148_0014.JPG  異常なし  異常なし/148_0014.JPG   
5   113_0096.JPG  異常なし  異常なし/113_0096.JPG   
6   184_0171.JPG  異常なし  異常なし/184_0171.JPG   
7   105_0045.JPG  異常なし  異常なし/10

Train on 1664 samples, validate on 46 samples
Epoch 1/10
 - 2s - loss: 8.0596 - acc: 0.4994 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 9/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 10/10
 - 1s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Test accuracy: 0.5
Train on 1664 samples, validate on 46 samples
Epoch 1/10
 - 1s - loss: 7.9986 - acc: 0.5024 - val_loss: 